In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

app_data = pd.read_csv('googleplaystore.csv')

In [10]:
def missing_values(test):
    total = test.isnull().sum().sort_values(ascending=False)
    percent_1 = test.isnull().sum() / test.isnull().count() * 100
    percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
    
    return missing_data.head((missing_data['Total'] > 0).sum())


In [11]:
# Display the amount of missing values per column
# and what percentage of the column contains missing values
missing_values(app_data)

,Total,%
Rating,1474,13.6
Current Ver,8,0.1
Android Ver,3,0.0
Content Rating,1,0.0
Type,1,0.0


In [12]:
# Removing app titled Life Made Wifi Touchscreen Photo Frame
# Reason: many missing and unusable values
# app_data = app_data.drop(10472)

# Removing rows with missing rating 
# I don't think I did anything with this line -Ilia
app_data = app_data[app_data.Rating.notnull()]


In [13]:
# Count unique values in App and see that there are duplicates
unique_apps = app_data["App"].nunique()
all_apps = app_data["App"].size

f"total app count: {all_apps}, unique app count: {unique_apps}"

'total app count: 9367, unique app count: 8197'

In [14]:
# Removing duplicate apps
app_data = app_data.drop_duplicates(subset = "App")

new_all_apps = app_data["App"].size
f"total app count after dropping duplicates: {new_all_apps}"

'total app count after dropping duplicates: 8197'

In [15]:
# Create df for Rating
# the y df
app_data_x = app_data["Rating"]

# Create df for the Rest of the features
# the x df
columns_drop = ["Rating", "App"]
app_data_y = app_data.drop(columns_drop, axis = 1)

In [28]:
# Convert Installs to numeric

app_data_y["Installs"].dtype
app_data_y["Installs"].describe

# idk why the line below is giving an error -Ilia
# app_data_y["Installs"] =  pd.to_numeric((app_data_y["Installs"].str.replace("+", "")).str.replace(",", ""))



<bound method NDFrame.describe of 0            10,000+
1           500,000+
2         5,000,000+
3        50,000,000+
4           100,000+
            ...     
10834           500+
10836         5,000+
10837           100+
10839         1,000+
10840    10,000,000+
Name: Installs, Length: 8197, dtype: object>

In [18]:
#See all unique values in the Android version column
app_data_y["Android Ver"].unique()
# Note that most are num and up. Hence, we can split the column into 3 by space

#Take the first part of the android version
app_data_y["Android Ver"] = app_data["Android Ver"].str.split(expand = True)

In [19]:
# There are 3 values in type column: free, paid and nan. There is only 1 nan so I looked at it and
# since the price is set to 0, I will put free for that cell
app_data_y["Type"][9148] = "Free"

# Change price column to free = 0 and paid = 1
paid_types = app_data_y["Type"].unique()

In [20]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
app_data_y['Type'] = labelencoder.fit_transform(app_data_y['Type'])

In [21]:
# Select numeric columns that can be converted as is
num_cols = ["Reviews"]
#test